In [2]:
import GPUtil


In [1]:
%run -i 'vcutils/taggerhelperfun.py'

In [2]:
%run -i 'vcutils/C__flaskdisplay.py'

In [3]:
fd=C__flaskdisplay('001')

In [19]:
yolo_hncdstop=YOLO('HNCDS/weights/HNCDS.weights')
yolo_hncdsside=YOLO('HNCDS/weights/HNCDSside.weights')

In [18]:
inception=Inception('HNCDS/weights/HNCDS.pb')

In [27]:
progress=Progress()
cap=cv2.VideoCapture('HNCDS/sample/2.mp4')
i=0
while 1:
    ret,frame=cap.read()
    if not ret:break
    i+=1
    if i%10!=0:continue        
    framecopy=frame.copy()
    res=yolo_hncdsside.infer(frame)
    zipres=list(x for x in zip(*res))
    for label,prob,rect in zipres:
        x1,y1,x2,y2=C__rect(rect).x1y1x2y2()
        if label=='p':
            progress.print(inception.infer(frame[y1:y2,x1:x2]))
    Tagger.yolodrawres(frame,list(zip(*zipres)),yolo_hncdsside.classes)
    fd.imswk(frame,1)

('p', 0.8622997)
('p', 0.8564898)
('p', 0.76347184)
('p', 0.9538895)
('n', 0.90462136)


In [25]:
progress=Progress()
cap=cv2.VideoCapture('HNCDS/sample/1.mp4')
i=0
while 1:
    ret,frame=cap.read()
    if not ret:break
    i+=1
    if i%10!=0:continue        
    framecopy=frame.copy()
    res=yolo_hncdstop.infer(frame)
    zipres=list(x for x in zip(*res))
    for label,prob,rect in zipres:
        x1,y1,x2,y2=C__rect(rect).x1y1x2y2()
        if label=='p':
            progress.print(inception.infer(frame[y1:y2,x1:x2]))
    Tagger.yolodrawres(frame,list(zip(*zipres)),yolo_hncdstop.classes)    
    fd.imswk(frame,1)

('n', 0.5962067)
('n', 0.5994929)
('p', 0.8959725)
('p', 0.5771009)
('p', 0.9430079)


In [13]:
yolo_tcdsnp=YOLO('TCDS/weights/TCDSnp.weights')

In [15]:
cap=cv2.VideoCapture('TCDS/sample/1.mp4')
while 1:
    ret,frame=cap.read()    
    Tagger.yolodraw(frame,yolo_tcdsnp)
    fd.imswk(frame,1)

KeyboardInterrupt: 

In [ ]:
def maskoutcontainer(image0,maskout=True):
    from Utils.helper import procimage
    r0 = procimage('clpsmaskrcnn',image0)
    if 0:visualize.display_instances(
                    image0, r0['rois'], r0['masks'], r0['class_ids'],
                    VOC_CLASSES, r0['scores'],
                    show_bbox=True, show_mask=True,
                    title="Predictions")
    elif 0:
        visualize.display_instances(
            image0, r0['rois'], r0['mask'], r0['class'],
            VOC_CLASSES, r0['scores'],
            show_bbox=True, show_mask=True,
            title="Predictions")
    newmask=np.zeros(image0.shape[:2],dtype=bool)
    classids=r0['class_ids']
    for maski in range(len(r0['class_ids'])):
        if classids[maski]==1:
            mask=r0['masks'][:,:,maski]
            for i in range(mask.shape[1]):
                try:
                    newmask[0:np.max(np.argwhere(mask[:,i]==1))+10,i]=1
                except:
                    pass
    #newmask[:,0:np.min(np.argwhere(newmask),axis=0)[1]]=1
    #newmask[:,np.max(np.argwhere(newmask),axis=0)[1]:]=1
    newmask[:,np.max(newmask,axis=0)==False]=1
    for maski in range(len(r0['class_ids'])):
        if classids[maski]!=1:
            mask=r0['masks'][:,:,maski]
            newmask[mask]=1    
    if maskout:
        image0[newmask]=0
    return newmask

def overlaymask(mask,image,alpha=0.5):
    overlay = image.copy()
    output = image.copy()
 
    # draw a red rectangle surrounding Adrian in the image
    # along with the text "PyImageSearch" at the top-left
    # corner
    overlay[mask]=(255,0,0)    
    cv2.addWeighted(overlay, alpha, output, 1 - alpha,0, output)
    return output
l__imf=sorted(glob.glob('CLPS/sample/2020-01-09/15-00-29/raw/t*.jpg'))
for imf in l__imf:
    im=cv2.imread(imf)
    newmask=maskoutcontainer(im, False)
    im=overlaymask(newmask,im)
    #print(imf,procimage('clpsmaskrcnn',im))
    fd.imswk(im,1)

In [30]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image

# Initialize PaddleOCR model
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Use English model
pass

[2025/04/22 22:14:50] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/hostuser/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/hostuser/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch

In [43]:
im=cv2.imread('PMNRS/sample/1.jpg')
imcp=im.copy()
res=(yolo_hncdstop.infer(im))
print(res)
for label,prob,rect in zip(*res):
    if label=='t':
        x1,y1,w,h=rect
        x2,y2=x1+w,y1+h
        pass
imcrop=im[y1:y2,x1:x2]
img_rgb = cv2.cvtColor(imcrop, cv2.COLOR_BGR2RGB)
#Tagger.yolodrawres(imcp,res,hncdsyolo.classes)
#fd.imswk(imcp,1);
result = ocr.ocr(img_rgb, cls=True)

print(result)        
print("HERE")
# Display recognized text
for line in result[0]:
    box, (text, confidence) = line
    print(f"Detected Text: {text}, Confidence: {confidence:.2f}")

# Optional: visualize
#image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result[0]]
txts = [line[1][0] for line in result[0]]
scores = [line[1][1] for line in result[0]]
im_show = draw_ocr(img_rgb, boxes, txts, scores, font_path='fonts/simfang.ttf')
im_show = Image.fromarray(im_show)
im_show.save('/tmp/result.jpg')
imout=cv2.imread('/tmp/result.jpg')
fd.imswk(imout,1)

(['t'], [0.99843836], array([[206,   5, 226, 162]], dtype=int32))
[2025/04/22 22:21:32] ppocr DEBUG: dt_boxes num : 4, elapsed : 0.01659989356994629
[2025/04/22 22:21:32] ppocr DEBUG: cls num  : 4, elapsed : 0.007981538772583008
[2025/04/22 22:21:32] ppocr DEBUG: rec_res num  : 4, elapsed : 0.07406878471374512
[[[[[1.0, 0.0], [36.0, 0.0], [36.0, 11.0], [1.0, 11.0]], ('14:32', 0.9057049751281738)], [[[75.0, 31.0], [151.0, 31.0], [151.0, 80.0], [75.0, 80.0]], ('914', 0.9987166523933411)], [[[38.0, 85.0], [186.0, 83.0], [187.0, 111.0], [39.0, 114.0]], ('XD7687S', 0.9636298418045044)]]]
HERE
Detected Text: 14:32, Confidence: 0.91
Detected Text: 914, Confidence: 1.00
Detected Text: XD7687S, Confidence: 0.96


-1